<hr style="height: 1px;">
<i>This notebook was authored by the 8.S50x Course Team, Copyright 2022 MIT All Rights Reserved.</i>
<hr style="height: 1px;">
<br>

<h1>Lesson 20: Monte Carlo I</h1>

<a name='section_21_0'></a>
<hr style="height: 1px;">


## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L20.0 Overview</h2>


<h3>Navigation</h3>

<table style="width:100%">
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_21_1">L20.1 Monte Carlo Computations</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#exercises_21_1">L20.1 Exercises</a></td>
    </tr>
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_21_2">L20.2 Monte Carlo Diffusion</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#exercises_21_2">L20.2 Exercises</a></td>
    </tr>
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_21_3">L20.3 Adding Elements of Realism</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#exercises_21_3">L20.3 Exercises</a></td>
    </tr>
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_21_4">L20.4 Modeling Physics Observables: Bragg Scattering for Proton Therapy</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#exercises_21_4">L20.4 Exercises</a></td>
    </tr>
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_21_5">L20.5 Bragg Scattering for Proton Therapy in Multiple Dimensions</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#exercises_21_5">L20.5 Exercises</a></td>
    </tr>
</table>

<h3>Learning Objectives</h3>

We have been randomly sampling and building toys throughout this class, both of which are elements of Monte Carlo (MC) simulations. However, we have yet to devote a whole Lesson to MC simulation itself. In the next two Lessons, we are going to build a MC simulation similar to one that is critical for proton medical therapy. Then, we are going to explore Machine Learning MC strategies, and show how some of the biggest advances in Artificial Intelligence are changing the way we simulate and model scientific data.

<h3>Slides</h3>

You can access the slides related to this lecture at the following link: <a href="https://github.com/mitx-8s50/slides/raw/main/module3_slides/L21_slides.pdf" target="_blank">L20 Slides</a>

<h3>Installing Tools</h3>

Before we do anything, let's make sure we install the tools we need.

In [ ]:
#>>>RUN: L20.0-runcell00

!pip install pylandau #from here: https://pypi.org/project/pylandau/
!pip install git+https://github.com/SengerM/landaupy

#from here: https://github.com/SengerM/landaupy
#https://github.com/SengerM/landaupy/blob/main/LICENSE

<h3>Importing Libraries</h3>

Before beginning, run the cell below to import the relevant libraries for this notebook. 

In [ ]:
#>>>RUN: L20.0-runcell01

import imageio
from PIL import Image
import pylandau
from landaupy import landau

import numpy as np
import matplotlib.pyplot as plt
import csv
import math
from scipy import optimize as opt 
from scipy.integrate import quad
import random

<h3>Setting Default Figure Parameters</h3>

The following code cell sets default values for figure parameters.


In [ ]:
#>>>RUN: L20.0-runcell02

#set plot resolution
%config InlineBackend.figure_format = 'retina'

#set default figure parameters
plt.rcParams['figure.figsize'] = (9,6)

medium_size = 12
large_size = 15

plt.rc('font', size=medium_size)          # default text sizes
plt.rc('xtick', labelsize=medium_size)    # xtick labels
plt.rc('ytick', labelsize=medium_size)    # ytick labels
plt.rc('legend', fontsize=medium_size)    # legend
plt.rc('axes', titlesize=large_size)      # axes title
plt.rc('axes', labelsize=large_size)      # x and y labels
plt.rc('figure', titlesize=large_size)    # figure title

<a name='section_21_1'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L20.1 Monte Carlo Computations</h2>  

| [Top](#section_21_0) | [Previous Section](#section_21_0) | [Exercises](#exercises_21_1) | [Next Section](#section_21_2) |

*The material in this section is discussed in the video **<a href="https://courses.mitxonline.mit.edu/learn/course/course-v1:MITxT+8.S50.3x+3T2023/block-v1:MITxT+8.S50.3x+3T2023+type@sequential+block@seq_LS20/block-v1:MITxT+8.S50.3x+3T2023+type@vertical+block@vert_LS20_vid1" target="_blank">HERE</a>.** You are encouraged to watch that video and use this notebook concurrently.*


<h3>Numerical Integration with Monte Carlo</h3>

You have probably noticed that in many cases where I was finding averages, standard deviations, and other moments of distributions, I just computed the means of the moments using sampled distributions. I did what is referred to as Monte Carlo Integration (or more generally bootstrapping), where I just sampled a distribution and integrated by sampling. Namely,

$$
\begin{equation}
E[x^{n}p(x)] = \int_{-\infty}^{\infty} x^{n} p(x) dx = \frac{1}{N}\sum_{i=1}^{N} x_{i}^{n}
\end{equation}
$$

where here the $x_{i}\in p(x)$ are sampled from the probability distribution function. So, what happens when we have a distribution but we don't know its analytic form. How can we sample it?  

There are a lot of ways to do this, perhaps the most well known is Markov Chain Monte Carlo (MCMC). However, the simplest is to just turn our distribution into a 2D image and randomly sample points on the image. Instead of writing the points out, **let's just do it.**


<h3>An Example</h3>

To see how this works, let's walk through a calculation of integrating the area of a quarter circle of radius 1. We know that the integral is given by $A={\pi}/{4}$, so we can check our math. What we will do is

1. Sample randomly in x from 0 to 1
2. Sample randomly in y from 0 to 1
3. Check to see if x and y are within our quarter circle.
4. Compute the ratio of the number of samples within our quarter circle divided by the total number of points.
5. Since we know the area of the full sample is exactly 1, this ratio is the area we are trying to find.

Let's look at the code. Note that since we are randomly sampling, our uncertainty on our random sample will just be the Poisson uncertainty or $\frac{A}{\sqrt{N_{\rm samples}}}$. Try varying the number of samples to see how the answer and its uncertainty change. Does it look like the Poisson uncertainty gives a good approximation to the deviations from the exact result?

In [ ]:
#>>>RUN: L20.1-runcell01

#First let's just compute the area of a quarter circle with radius 1
def quarterarea(iN):
    area=0
    lXin = np.array([])
    lYin = np.array([])
    lXout = np.array([])
    lYout = np.array([])
    for i0 in range(iN):
        #Sample X and Y
        pX = np.random.uniform(0,1)
        pY = np.random.uniform(0,1)
        #Check if its radius is in 1
        if np.sqrt(pX**2+pY**2) < 1:
            lXin = np.append(lXin,pX)
            lYin = np.append(lYin,pY)
            area += 1 # count it
        else:
            lXout = np.append(lXout,pX)
            lYout = np.append(lYout,pY)
    print(area)
    return (float(area)/float(iN)),lXin,lYin,lXout,lYout

#sample points
lN=1000
#lN=100000
a,lXin,lYin,lXout,lYout=quarterarea(lN)
print("Pi (4*area) :",f"{a*4:.6f}","+/-",f"{4*a/math.sqrt(lN):.6f}") #gotta put an uncertainty
print("Exact answer:",f"{np.pi:.6f}","\nRatio - 1:",f"{(a*4/np.pi)-1.0:.6f}","+/-",f"{4*a/(np.pi*math.sqrt(lN)):.6f}")

#Make the plots square so the quarter circle looks like a circle
plt.rcParams['figure.figsize'] = (7,7)

plt.plot(lXin,lYin,marker='.',linestyle = 'None')
plt.plot(lXout,lYout,marker='.',linestyle = 'None')
plt.show()

#Revert to default plot aspect ratio
plt.rcParams['figure.figsize'] = (9,6)

The idea with Monte Carlo integration is that we calculate an integral by evaluating the function. We don't actually have to compute the integral. This avoids what is potentially a very complicated step.  As you probably know well, computing integrals can be very hard, but this way gets at our answer just through functional evaluation.

Now, let's compute the integral of some arbitrary function $y=f(x)$. As we did with our circle, we can compute the integral by plotting this function over a range. Here, we will use a range $-6 < x < 3$. From this, what we will do is sample over a uniform distribution given by the $x$ and $y$ ranges. However, how do we know what range in $y$ to use? The option used in this example is to use a minimization function `opt.minimize_scalar`. To use this same function to find the maximum, we simply find the minimum of the negative of the function.

In [ ]:
#>>>RUN: L20.1-runcell02


#use this random seed
np.random.seed(10)

#Now let's consider integrating some random function
def f(x):
    return x**4 + 3*(x-2)**3 - 15*(x)**2 + 1

#Now let's multiply it by -1 to make the range calculation fast
def fneg(x):
    return -1*(x**4 + 3*(x-2)**3 - 15*(x)**2 + 1)


#First thing is to define a range in x
xmin=-6
xmax=3
x = np.linspace(xmin, xmax, 100)
plt.plot(x, f(x));
#plt.show()


#Now we need to find a range in y
sol=opt.minimize_scalar(f,bounds=(xmin, xmax))#, method='Brent')
ymin=sol.fun
#y-max is to get the minimum of negative f
sol=opt.minimize_scalar(fneg,bounds=(xmin, xmax))#, method='Brent')
ymax=-1*sol.fun

print('[ymin,ymax]:', ymin, ymax)

"""
#Now we need to find a range in y
ymin=min(f(x))
ymax=max(f(x))
print('[ymin,ymax]:', ymin, ymax)
"""

lN=10000
#now, let's sample a 2D grid y-min and y-max and compute the integral
lXin = np.array([])
lYin = np.array([])
lXout = np.array([])
lYout = np.array([])
for i0 in range(lN):
    #Try a uniform distribution
    pX = abs(xmax-xmin)*np.random.uniform(0,1)+xmin
    pY = abs(ymax-ymin)*np.random.uniform(0,1)+ymin
    #Try a normal distribution
    #pX = abs(xmax-xmin)*np.random.normal(0,1)+xmin
    #pY = abs(ymax-ymin)*np.random.normal(0,1)+ymin
    pYMin = f(pX)
    if pY < pYMin:
        lXin = np.append(lXin,pX)
        lYin = np.append(lYin,pY)
    else:
        lXout = np.append(lXout,pX)
        lYout = np.append(lYout,pY)


plt.plot(lXin,lYin,marker='.',linestyle = 'None', color='orange')
plt.plot(lXout,lYout,marker='.',linestyle = 'None', color='green')
plt.axvline(sol.x, c='red', lw=3)
plt.plot(x, f(x), 'b-', lw=3)
#plt.ylim(-800,0)
#plt.xlim(-6,3)
plt.show();

From the plot generated by this scenario, we can see that the orange points are below the line from roughly -800 to a little below 0, and the green points are above the line. We can make a histogram of the fraction of the points which are above and below the line for bins in $x$.

In [ ]:
#>>>RUN: L20.1-runcell03

_,bins,_= plt.hist(lXin,bins=10,alpha=0.5,density=True, label='below') #blue in histogram
plt.hist(lXout,alpha=0.5,bins=bins,density=True, label='above') #orange in histogram
#plt.xlim(-6,3)
plt.legend(loc=1)
plt.show();

print("number below:",len(lXin),"number above:",len(lXout))

Note that a uniform distribution in the variable of integration (the $x$ values in the example above) may not always be the best choice. However, we have to be careful with this method of finding an integral if we are sampling, for example, a Gaussian distribution. In that case, our sampling would be biased by the Gaussian we chose, which means our integral will not be correct. We can write this out as the product of our function times the phase space we are sampling. In this case, the phase space is given mutliplying by Guassian weighted pdf $p_{G}(x,y)$:


$$
\begin{equation}
\mathcal{I}=\int_{\rm space} f(x,y) p_{G}(x,y) dx dy
\end{equation}
$$


The procedure described above is known as "Area-based" sampling, and is considered a method of Monte-Carlo Integration, which is a very rich field. In fact, all high energy physics simulations are based on it. Basically, the function we sample from starts with a collision found using the probability distribution of a set of possible collision outcomes. We then proceed to put the particles generated in this collision through a point by point simulation, wherein each particle is followed as it go through all the elements of a particular detector. At each step, the probability of each particle to interact with, or leave a signal in, the detector material is sampled. Finally, we aggregate our distributions based on the full set of detector responses. We will discuss the usefulness of Monte Carlo Simulation in more detail later on.


<a name='exercises_21_1'></a>     

| [Top](#section_21_0) | [Restart Section](#section_21_1) | [Next Section](#section_21_2) |


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Ex-20.1.1</span>


How would you compute the integral of the function given in code cell `L20.1-runcell02` over the range `[-6,3]`? Select the correct answer below, where the numbers above and below the line refer to the output of code cell `L20.1-runcell03`. Think carefully about how one might need to consider negative signs in each case.


A) The integral is equal to the total area sampled times the fraction of the points above the line defined by our function.\
B) The integral is the negative of the quantity given in option A.\
C) The integral is equal to the total area sampled times the fraction of the points below the line defined by our function.\
D) The integral is the negative of the quantity given in option C.\
E) To calculate the integral using the number of points above and below the line found by code cell `L20.1-runcell03`, corrections need to be made for the fact that $y_{max}$ is not zero.\
F) To calculate the integral using the number of points above and below the line found by code cell `L20.1-runcell03`, corrections need to be made for the region with negative $x$ values.\
G) To calculate the integral using the number of points above and below the line found by code cell `L20.1-runcell03`, corrections need to be made for the issues mentioned in both options E and F.


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Ex-20.1.2 </span>

Calculate the integral of this function using the Monte Carlo method outlined above, and report your answer as a number with precision `1e-2`. In order to compare your answer to ours, use the same conditions as defined above, namely:

<pre>
#use this random seed
np.random.seed(10)

#use this array of x-values
xmin=-6
xmax=3
x = np.linspace(xmin, xmax, 100)

#use this number of points
lN=10000
</pre>

<b>Remember to correctly apply a negative sign, if applicable!</b>

How does your numerically calculated value compare to the expected value? You can explicitly solve the integral mathematically, or use a built-in `scipy` integration method (e.g., `scipy.integrate.quad`) to compare to your Monte Carlo integration.

Bonus: Write a Monte Carlo integrator that computes the definite integral of any generic function.

In [ ]:
#>>>EXERCISE: L20.1.2

#define the fraction of points above the function
frac_above = None

#define the total area of the integration region,
#i.e., where the points are distributed
area = None

#define an adjustment to the total area
#since the function in this range is
#completely below the y-axis
adjust = None

#define the integral using the terms above
MC_integal =  None

print('Monte Carlo integration:', MC_integal)

#COMPARE TO PRE-DEFINED NUMERICAL INTEGRATOR
# call quad to integrate f from -6 to 3
builtin_int = None

print('Numerical integration:', builtin_int)
print('Ratio of MC/exact:',MC_integal/builtin_int)

### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Ex-20.1.3 </span>

To get a more accurate answer using the Monte Carlo integration method that we just explored, which of the following approaches could you implement? Select ALL that apply.

A) Increase the number of random points sampled.\
B) Average over several different trials, using different starting random seeds.\
C) Modify the code to generate more samples in the regions where the function varies most steeply.


<a name='section_21_2'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L20.2 Monte Carlo Diffusion</h2>  

| [Top](#section_21_0) | [Previous Section](#section_21_1) | [Exercises](#exercises_21_2) | [Next Section](#section_21_3) |

*The material in this section is discussed in the video **<a href="https://courses.mitxonline.mit.edu/learn/course/course-v1:MITxT+8.S50.3x+3T2023/block-v1:MITxT+8.S50.3x+3T2023+type@sequential+block@seq_LS20/block-v1:MITxT+8.S50.3x+3T2023+type@vertical+block@vert_LS20_vid2" target="_blank">HERE</a>.** You are encouraged to watch that video and use this notebook concurrently.*

<h3>Overview</h3>

Another system that can be simulated with Monte Carlo techniques is a collection of particles undergoing random walks. To start with, let's consider a simple diffusion model. This works by considering a particle moving through some sort of fluid (i.e., a collection of other particles) with velocity $\vec{v}$ in a specific direction. We assume that it will move a distance $\ell$ and then scatter off one of the particles in the fluid. The scattering will cause the direction of the first particle's velocity to change in a random direction. The angular spread of the deflection depends on the velocity (or equivalently the average energy) of the particles in the fluid, which could be, for example, the atmosphere.

To build a simple model, let's consider just Brownian motion, where we simulate each timestep and give the particle a random kick corresponding to a collision with a single other particle. Thus, we can write this as randomly choosing an angle every time step, and simulating the particle motion.

For each particle, we will step the particle 100 times. Furthermore, to show some intelligent thought about parallel computing, we are going to do this simultaneously for 1000 particles at a time. That means we will make an array of 1000 particles and run the simulation simultaneously using array manipulations as we have done previously.

We start each particle at the origin with a Gaussian distributed velocity. We give the particles it will scatter off (which we assume are the same mass) the same Gaussian distribution of velocities. We assume that the scatterings are elastic, which is most easily handled by going to the center of mass frame of the two particles. In this frame, the magnitudes of the velocities of the two particles, which are equal and opposite velocities don't change. Instead, the two particle emerge deflected by a random angle uniformly distributed over $2\pi$.

Note, in the last little bit of the code below, we will make a rotation matrix that we tile (i.e., we create an array of matrices). Then, we apply the rotation matrices to our velocities using the following Einstein summation notation.

$$
v^{u}_{j} =  R^{u}_{\theta,ij} \left(v^{u}_{i}\right)
$$

In [ ]:
#>>>RUN: L20.2-runcell01

def MCSim(sigmav=1,nsteps=10000,ntoys=5000,dt=0.01,iDump=True):
    partpos=np.zeros((ntoys,2)) #x and y
    partvel=np.random.normal(0,1,(ntoys,2)) #let's assume all particles are randomly distributed
    for t0 in range(nsteps):
        partpos += partvel*dt
        #now at each step we will assume the particle collides with another
        opartvel = np.random.normal(0,1,(ntoys,2)) #Sample the "other" particles momentum
        comvel   = 0.5*(partvel + opartvel) #Now compute the comoving velocities in the COM frame
        difvel   = 0.5*(partvel - opartvel) #Now compute the collision velocity in the COM frame
        theta = np.random.uniform(0,2.*np.pi,ntoys)#elastic collisions in the COM equate to equal energy ad a theta rotation
        R     = np.array(((np.cos(theta), -1*np.sin(theta)), (np.sin(theta), np.cos(theta))))
        #shape things so they are right
        RMat  = np.swapaxes(R,0,2)
        tmp=np.einsum('BNi,Bi ->BN', RMat, difvel) #Now rotate the colliding velocities over all toys
        partvel = comvel+tmp

    if iDump:
        plt.hist(np.sqrt(partpos[:,0]**2+partpos[:,1]**2))
        plt.xlabel("$\sqrt{\Delta x^2 + \Delta y^2}$")
        plt.ylabel("toys")
        print("Mean distance from the starting point",np.mean(np.sqrt(partpos[:,0]**2+partpos[:,1]**2)))
        print("Standard deviation of the distance in x:",np.std(partpos[:,1]),"y:",np.std(partpos[:,0]))
        print("Mean magnitude of the velocity",np.mean(np.sqrt(partvel[:,0]**2+partvel[:,1]**2)))
        plt.show()
        plt.hist(partvel[:,0])
        plt.xlabel("$v_{x}$")
        plt.ylabel("toys")
        plt.show()

#MCSim(nsteps=1) #perform only one step
MCSim() #perform default number of steps

Now, we can compare these numbers with our knowledge of Brownian motion, noting how things diffuse. We would like to confirm our measurement of the RMS $x_{\rm rms}=\sqrt{\langle \bar{x}^2 \rangle}$.

We can think of this as the width of the distribution after $N$ collisions with a collision length of $\ell$. For a time $t$ and a velocity $v$ the number collisions will be $N=(vt)/\ell$ or, in other words:

$$
x_{\rm rms} = \sqrt{N}\ell = \sqrt{vt\ell}
$$

Anther way of thinking of this is in terms of the velocity:

$$
\frac{d}{dt}\langle \bar{x}^2 \rangle = 2\bar{x}\cdot\frac{d \bar{x}}{dt} = 2\langle \bar{x}\cdot\bar{v} \rangle\approx2\langle v^2\rangle t
$$

You can read more discussion of Brownian motion in Section 5 of this <a href="https://scholar.harvard.edu/files/schwartz/files/2-diffusion.pdf">article</a>. What this means is that we can exactly predict the rate of diffusion analytically and compare to the output of our MC simulation. Let's go ahead and do that.

First, the average total velocity in two dimensions can be defined as $\langle\sqrt{v_{x}^2+v_{y}^2}\rangle$. Since $v_{x}$ and $v_{y}$ are both Gaussian distributions, the sum of the squares of these distributions is known as a <a href="https://en.wikipedia.org/wiki/Rayleigh_distribution">Rayleigh distribution</a> ($f(x,\sigma)$), which is  just a $\chi^{2}_{2}$ distribution of 2 degrees of freedom with an additional scale term.

$$
\langle\sqrt{v_{x}^2+v_{y}^2}\rangle = \langle f(x,\sigma_{v_{i}}) \rangle = \sigma_{v_{i}}\sqrt{\frac{\pi}{2}}
$$

From the above, we can immediately calculate the average RMS of $x$ for a single collision. Note that our simulation forces the distance between collisions to be $\ell=2vdt$, where the factor of 2 comes from the fact that both particles are moving towards each other:

$$
\langle \ell \rangle =  \langle 2 v dt \rangle\\
x_{\rm rms} = \sqrt{N}\langle\ell\rangle= \sqrt{N}\sqrt{2\langle v^2\rangle} dt = \sqrt{N2\frac{\pi}{2}}dt
$$


 Since the velocity Gaussians in our specific model both have  $\sigma_{v_{i}}=1$, we have:

$$
x_{\rm rms} = \sqrt{2\sigma^{2}_{v_{i}}\frac{\pi}{2}}\sqrt{N}dt=\sqrt{\pi N}dt
$$

Let's compute these values.

In [ ]:
#>>>RUN: L20.2-runcell02

print("Rayleigh",np.sqrt(np.pi/2))
print("Rayleigh dt sqrt(N pi)",np.sqrt(np.pi/2)*np.sqrt(2)*0.01*np.sqrt(10000))
print("Expected mean distance from the center: ",np.sqrt(np.pi/2)*np.sqrt(2)*0.01*np.sqrt(10000)*np.sqrt(np.pi/2))


These agree well with the MC values for average distance $x\approx 2.2$ and its standard deviation $x_{rms}\approx 1.7$.

<a name='exercises_21_2'></a>     

| [Top](#section_21_0) | [Restart Section](#section_21_2) | [Next Section](#section_21_3) |


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Ex-20.2.1</span>

What happens to the velocity distribution as we run our simulation over many time steps? Complete the function below, which is just a slight change to the function `MCSim` above, to compare the total velocity before and after, then choose the best option below:

A) There is at most only a very small change which is not statistically significant.\
B) After a long time, the velocity decreases\
C) After a long time, the velocity increases.

In [ ]:
#>>>EXERCISE: L20.2.2

def MCSim_compare(sigmav=1,nsteps=10000,ntoys=5000,dt=0.01,iDump=True):
    partpos=np.zeros((ntoys,2)) #x and y
    partvel=np.random.normal(0,1,(ntoys,2))
    
    print("velocity before:", #YOUR CODE HERE)
    
    #YOUR CODE HERE
    #HINT: see function in `L20.2-runcell01`
   
    print("velocity after:", #YOUR CODE HERE)

MCSim()

### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Ex-20.2.2</span>

Consider a microscopic particle undergoing Brownian motion in a fluid. The particle's displacement at each time step follows a random walk pattern due to collisions with surrounding molecules. Which of the following statements about Brownian motion and random walks is/are correct? Select ALL that apply:

A) Brownian motion follows a deterministic trajectory governed by classical mechanics, and the particle's position at any given time can be precisely predicted.

B) In Brownian motion, the particle's path is continuous, smooth, and follows a well-defined trajectory over time.

C) The increments of a random walk in Brownian motion are often modeled as independent, identically distributed Gaussian random variables.

D) Brownian motion violates the conservation of energy principle, leading to erratic behavior that contradicts classical physics.

<a name='section_21_3'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L20.3 Adding Elements of Realism</h2>  

| [Top](#section_21_0) | [Previous Section](#section_21_2) | [Exercises](#exercises_21_3) | [Next Section](#section_21_4) |

*The material in this section is discussed in the video **<a href="https://courses.mitxonline.mit.edu/learn/course/course-v1:MITxT+8.S50.3x+3T2023/block-v1:MITxT+8.S50.3x+3T2023+type@sequential+block@seq_LS20/block-v1:MITxT+8.S50.3x+3T2023+type@vertical+block@vert_LS20_vid3" target="_blank">HERE</a>.** You are encouraged to watch that video and use this notebook concurrently.*

Now, starting from our initial idealized Monte Carlo simulation, we can start to add elements of realism which will, of course, inevitably will make our MC code slower. As a first example, let's put a wall in our particle simulation. When a particle hits the wall, we will need to invert the component of the velocity perpendicular to the wall.

Let's go ahead and implement this. If the x-position of our sample is greater than the wall value ($x=2$ in the code below) then,  instead of simulating an elastic collision, we simply flip the $x$ component of the velocity. 

In [ ]:
#>>>RUN: L20.3-runcell01

def MCSim(sigmav=1,nsteps=10000,ntoys=5000,dt=0.01,iDump=True):
    partpos=np.zeros((ntoys,2)) #x and y
    partvel=np.random.normal(0,1,(ntoys,2))
    for t0 in range(nsteps):
        opartvel = np.random.normal(0,1,(ntoys,2))
        partpos += partvel*dt
        comvel   = 0.5*(partvel + opartvel)
        difvel   = 0.5*(partvel - opartvel)        
        theta = np.random.uniform(0,2.*np.pi,ntoys)
        R     = np.array(((np.cos(theta), -1*np.sin(theta)), (np.sin(theta), np.cos(theta))))
        RMat  = np.swapaxes(R,0,2)
        tmp=np.einsum('BNi,Bi ->BN', RMat, difvel)
        #Now, let's add our velocity if its not at the well
        partvel[partpos[:,0] < 2] = comvel[partpos[:,0] < 2]+tmp[partpos[:,0] < 2]
        #And let's flip it if it is at the wall        
        partvel[partpos[:,0] > 2] = [-1,1]*partvel[partpos[:,0] > 2]
        
    if iDump:
        _,bins,_=plt.hist(partpos[:,1],bins=30,alpha=0.5,label='y-position')
        plt.hist(partpos[:,0],bins=bins,alpha=0.5,label='x-position')
        plt.legend()
        plt.show()
        _,bins,_=plt.hist(partvel[:,1],bins=30,alpha=0.5,label='y-velocity')
        plt.hist(partvel[:,0],bins=bins,alpha=0.5,label='x-velocity')
        plt.legend()
        plt.show()
MCSim()

Now, we could do something even more complicated by imagining that we have a tube and we are looking at the Brownian motion within the tube. Particles which hit the tube will then have both the $x$ and $y$ components of their velocities reflected in the opposite direction. In other words, we have

$$
v_{i} = - v_{i} \forall i {\rm~ where~} r_{i}=\sqrt{x_{i}^2+y_{i}^2} > r_{\rm tube}
$$

or reflecting particles that try to escape the tube. Let's go ahead and make a tube of radius $2$.

In [ ]:
#>>>RUN: L20.3-runcell02

def MCSim_tube(sigmav=1,nsteps=10000,ntoys=5000,dt=0.01,rtube=2.0,iDump=True):
    partpos=np.zeros((ntoys,2)) #x and y
    partvel=np.random.normal(0,sigmav,(ntoys,2))
    for t0 in range(nsteps):
        opartvel = np.random.normal(0,sigmav,(ntoys,2))
        partpos += partvel*dt
        comvel   = 0.5*(partvel + opartvel)
        difvel   = 0.5*(partvel - opartvel)
        theta = np.random.uniform(0,2.*np.pi,ntoys)
        R     = np.array(((np.cos(theta), -1*np.sin(theta)), (np.sin(theta), np.cos(theta))))
        RMat  = np.swapaxes(R,0,2)
        #tmp2=np.matmul(RMat[0],difvel[0])
        tmp=np.einsum('BNi,Bi ->BN', RMat, difvel)
        #Now define inside the tube
        intube  = partpos[:,0]**2+partpos[:,1]**2 < rtube**2
        partvel[intube]  = comvel[intube]+tmp[intube]
        partvel[~intube] = -partvel[~intube]
    if iDump:
        _,bins,_=plt.hist(partpos[:,1],bins=30,alpha=0.5,label='y-position')
        plt.hist(partpos[:,0],bins=bins,alpha=0.5,label='x-position')
        plt.legend()
        plt.show()
        _,bins,_=plt.hist(partvel[:,1],bins=30,alpha=0.5,label='y-velocity')
        plt.hist(partvel[:,0],bins=bins,alpha=0.5,label='x-velocity')
        plt.legend()
        plt.show()
        
MCSim()

We can imagine considering even more complicated scenarios, and the nice thing is that all this really needs is for us to program it! It is also important here to note that we are taking advantage of the numpy scheme to parallelize everything. This allows us to simulate 1000s of particles at the same time, and makes everything nice and fast!

<a name='exercises_21_3'></a>     

| [Top](#section_21_0) | [Restart Section](#section_21_3) | [Next Section](#section_21_4) |


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Ex-20.3.1</span>

For particles in a tube, as simulated in code cell `L20.3-runcell02`, how do the spatial and velocity distributions of the particles change if we increase the velocity spread of the particles (in other words, if we increase `sigmav`)? Complete and run the code below to help answer this question. In order to make the tube less restrictive, set its radius to 10. Select ALL that apply.

Note: To ensure the same approximate number of collisions, one should appropriately increase the time step at lower velocities.

When the velocity spread is broader (i.e., larger `sigmav`),

A) the spatial distribution of particles becomes narrower.

B) the spatial distribution of particles remains the same.

C) the spatial distribution of particles becomes more wider.

D) the distribution of particle velocities becomes narrower.

E) the distribution of particle velocities remains the same.

F) the distribution of particle velocities becomes wider.


In [ ]:
#>>>EXERCISE: L20.3.1

print('sigmav=100')
#RUN `MCSim` with appropriate parameters
print()

print('sigmav=1')
#RUN `MCSim` with appropriate parameters

### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Ex-20.3.2</span>

What would the distribution of the particles look like if you had a box with walls at $x=\pm2$ and $y=\pm2$? Edit the `inbox` variables in the code below and run it to help answer this question.

A) The distribution is concentrated around the center of the square.

B) Particle density is higher at the corners of the square.

C) Particles are uniformly distributed in both dimensions.

D) Particles are more likely to be found near the edges of the square.


In [ ]:
#>>>EXERCISE: L20.3.2

def MCSim_box(sigmav=1,nsteps=10000,ntoys=5000,dt=0.01,iDump=True):
    partpos=np.zeros((ntoys,2)) #x and y
    partvel=np.random.normal(0,sigmav,(ntoys,2))
    for t0 in range(nsteps):
        opartvel = np.random.normal(0,sigmav,(ntoys,2))
        partpos += partvel*dt
        comvel   = 0.5*(partvel + opartvel)
        difvel   = 0.5*(partvel - opartvel)
        theta = np.random.uniform(0,2.*np.pi,ntoys)#elastic collisions in the COM equate to equal energy ad a theta rotation
        R     = np.array(((np.cos(theta), -1*np.sin(theta)), (np.sin(theta), np.cos(theta))))
        RMat  = np.swapaxes(R,0,2)
        tmp=np.einsum('BNi,Bi ->BN', RMat, difvel)

        inbox  = #YOUR CODE HERE
        inbox_x  = #YOUR CODE HERE
        inbox_y  = #YOUR CODE HERE
        
        partvel[inbox]  = comvel[inbox]+tmp[inbox]
        partvel[~inbox_x] = [-1.,1]*partvel[~inbox_x] 
        partvel[~inbox_y] = [1.,-1]*partvel[~inbox_y]
    if iDump:
        _,bins,_=plt.hist(partpos[:,1],bins=30,alpha=0.5,label='y-position')
        plt.hist(partpos[:,0],bins=bins,alpha=0.5,label='x-position')
        plt.legend()
        plt.show()
        _,bins,_=plt.hist(partvel[:,1],bins=30,alpha=0.5,label='y-velocity')
        plt.hist(partvel[:,0],bins=bins,alpha=0.5,label='x-velocity')
        plt.legend()
        plt.show()

MCSim_box()

<a name='section_21_4'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L20.4 Modeling Physics Observables: Bragg Scattering for Proton Therapy</h2>  

| [Top](#section_21_0) | [Previous Section](#section_21_3) | [Exercises](#exercises_21_4) | [Next Section](#section_21_5) |

*The material in this section is discussed in the video **<a href="https://courses.mitxonline.mit.edu/learn/course/course-v1:MITxT+8.S50.3x+3T2023/block-v1:MITxT+8.S50.3x+3T2023+type@sequential+block@seq_LS20/block-v1:MITxT+8.S50.3x+3T2023+type@vertical+block@vert_LS20_vid4" target="_blank">HERE</a>.** You are encouraged to watch that video and use this notebook concurrently.*


<h3>Overview</h3>

There are a broad range of physics processes that we can use Monte Carlo techniques to simulate. They typically involve scenarios where we don't have a good understanding of the analytic form of the whole system, but we understand how it works. Let's try an example by simulating a particle passing through matter. This is one of the most common simulations, because we understand all the different subprocesses well, but when we put them all together they are rather complicated.

We will do this in the context of proton therapy data. Proton therapy is a way to treat cancerous cells through a non-invasive approach whereby we fire energetic proton beams aimed at various body parts. These beams are precisely controlled to deliver a dose of radiation at just the right spot. Let's go ahead and do a simple simulation.


<h3>Sources</h3>

In what follows, we reference this paper (although the specific notation we use differs from that in the paper): https://pdg.lbl.gov/2022/reviews/rpp2022-rev-passage-particles-matter.pdf

The code we are using is from a "Toolkit for Monte Carlo simulation of ionizing radiation": https://github.com/nrc-cnrc/EGSnrc


<h3>The Model</h3>

What we are going to do first is model the energy loss (or mass stopping power) of a charged particle traveling through matter, which is given by the **Bethe-Bloch formula:**

$$
-\frac{dE}{dx} = \frac{4\pi nz^2}{m_{e}c^{2}\beta^{2}}\left(\frac{e^{2}}{4\pi\epsilon_{0}}\right)^{2}\left[\log\left(\frac{2\gamma^{2}m_{e}c^{2}\beta^{2}T_{\rm max}}{I(1-\beta)^{2}}\right)-\beta^{2}-\frac{\delta(\beta\gamma)}{2}\right]
$$

Let's break down the terms in this equation:

**1.** $-\dfrac{dE}{dx}$: The rate of energy loss per unit length (path length) of a charged particle as it moves through a material.

You might wonder why we don't write an equation for the change in energy $dE/dx$ and put the minus sign on the right to indicate that energy is lost. For obscure historical reasons, this somewhat odd looking notation is the conventional way that this equation is presented.
   
<br>

**2.** $\dfrac{4\pi nz^2}{m_{e}c^{2}\beta^{2}}\left(\dfrac{e^{2}}{4\pi\epsilon_{0}}\right)^{2}$: Various physical constants and characteristics of the particle and the material:
 - $n$ is the number density of electrons in the material.
 - $z$ is the charge of the particle.
 - $m_{e}$ is the rest mass of the electron.
 - $c$ is the speed of light.
 - $\beta$ is the velocity of the charged particle normalized to the speed of light ($\beta={v}/{c}$).
 - $e$ is the elementary charge (i.e. the magnitude of the charge of an electron or proton).
 - $\epsilon_{0}$ is the vacuum permittivity (the constant found in the equation for the force between two charged particles).


**3.** $\log\left(\dfrac{2\gamma^{2}m_{e}c^{2}\beta^{2}T_{\rm max}}{I(1-\beta)^{2}}\right)-\beta^{2}-\dfrac{\delta(\beta\gamma)}{2}$: The functional form describing the particle's energy loss processes, where:
 - $\gamma$ is the Lorentz factor $\left ({1}/{\sqrt{1-\beta^{2}}}\right)$ for the particle.
 - $T_{\rm max}$ is the maximum kinetic energy transfer in a single collision between the particle and an electron in the material. This can be calculated exactly for a 2-body elastic collision giving:
 $$T_{\rm max}=\frac{2m_{e}\left (c\beta\gamma\right )^2}{1+\dfrac{2\gamma m_{e}}{m}+\left(\dfrac{m_{e}}{m}\right )^2}$$
 with $m$ being the particle's mass.
 - $I$ is the mean ionization energy of the material.
 - $\delta(\beta\gamma)$ is a density correction term, accounting for the effects of atomic screening of the energy loss.


Now, to make a realistic numerical calculation, we need the value of the ionization energy, $I$, which you can find tabulated in numerous places. In particular, it is plotted as a function of the atomic number $Z$ of the material in Fig. 34.5 on page 9 of the <a href="https://pdg.lbl.gov/2022/reviews/rpp2022-rev-passage-particles-matter.pdf" target="_blank">paper</a> mentioned above. It depends on the atomic structure of each element, but is roughly flat as a function $Z$ as you get to heavier elements. For our purposes, we will just parameterize this using as table.

In addition, we also need the atomic masses $A$ of all the elements, which we can again take from various online sources. For people familiar with the MC code most often used to simulate physics detectors, we have taken these values from the particle simulation code Geant in Fortran. Let's plot these two quantities.

In [ ]:
#>>>RUN: L20.4-runcell01

#values
def I(iZ,iPlot=False):
    #https://github.com/nrc-cnrc/EGSnrc/blob/master/HEN_HOUSE/pegs4/pegs4.mortran#L1354-L1391
    lI=[19.2,41.8,40.,63.7,76.0,78.0,82.0,95.0,115.,137.,
     149.,156.,166.,173.,173.,180.,174.,188.,190.,191.,216.,233.,245.,
     257.,272.,286.,297.,311.,322.,330.,334.,350.,347.,348.,357.,352.,
     363.,366.,379.,393.,417.,424.,428.,441.,449.,470.,470.,469.,488.,
     488.,487.,485.,491.,482.,488.,491.,501.,523.,535.,546.,560.,574.,
     580.,591.,614.,628.,650.,658.,674.,684.,694.,705.,718.,727.,736.,
     746.,757.,790.,790.,800.,810.,823.,823.,830.,825.,794.,827.,826.,
     841.,847.,878.,890.,902.,921.,934.,939.,952.,966.,980.,994.]
    lZ=np.arange(1,len(lI)+1)
    if iPlot:
        plt.plot(lZ,lI/lZ)
        plt.xlabel('Z')
        plt.ylabel('I$_{adj}$/Z (eV/Z)')
        plt.show()
    return lI[iZ]*1e-6 #MeV not eV

def A(iZ,iPlot=False):
    #https://github.com/nrc-cnrc/EGSnrc/blob/master/HEN_HOUSE/pegs4/pegs4.mortran#L1354-L1391
    lA=[1.00797,4.0026,6.939,9.0122,10.811,12.01115,14.0067,
     15.9994,18.9984,20.183,22.9898,24.312,26.9815,28.088,30.9738,
     32.064,35.453,39.948,39.102,40.08,44.956,47.90,50.942,51.998,
     54.9380,55.847,58.9332,58.71,63.54,65.37,69.72,72.59,74.9216,
     78.96,79.808,83.80,85.47,87.62,88.905,91.22,92.906,95.94,99.0,
     101.07,102.905,106.4,107.87,112.4,114.82,118.69,121.75,127.60,
     126.9044,131.30,132.905,137.34,138.91,
     140.12,140.907,144.24,147.,150.35,151.98,157.25,158.924,162.50,
     164.930,167.26,168.934,173.04,174.97,178.49,180.948,183.85,
     186.2,190.2,192.2,195.08,196.987,200.59,204.37,207.19,208.980,
     210.,210.,222.,223.,226.,227.,232.036,231.,238.03,237.,242.,
     243.,247.,247.,248.,254.,253.   
    ]
    lZ=np.arange(1,len(lA)+1)
    if iPlot:
        plt.plot(lZ,lA/lZ)
        plt.xlabel('Z')
        plt.ylabel('A/Z (Atomic mass/Z)')
        plt.show()
    return lA[iZ-1]
lItmp=I(1,True)
lItmp=A(1,True)

Now we have all the elements to compute the Bethe-Bloch formula and look at charged particle energy loss over distance. For this part, we are going to focus on protons. However, this generally applies to a lot of different phenomena.  

Additionally, we will plot this for two atomic elements. Since we care about the human body, we will plot this for water (specifically the oxygen). In order to  show something heavier that's found in a human, we'll also calculate this for calcium, which is a major component of bone. More information about what a human body is comprised of can be found <a href="https://en.wikipedia.org/wiki/Composition_of_the_human_body" target="_blank">here.</a>

For comparison, we'll also show the energy loss for pions and muons. In all three cases, we'll plot the energy loss as a function of momentum.

You will note that the momentum axis is labelled "MeV" (million electron volts), which is actually a unit of energy. A more accurate notation with correct units is "MeV/$c$" (where $c$ is the speed of light). Physicists tend to get lazy about including factors of $c$. As another example, they also refer to the mass of particles using energy units (see the line `m_e = 0.511 # Mass of electron in MeV` in the code cell below), which omits the factor of $c^2$ that relates mass and energy in Einstein's famous equation $E=mc^2$.

In [ ]:
#>>>RUN: L20.4-runcell02

#https://indico.cern.ch/event/753612/contributions/3121551/attachments/1974578/3285956/MC_2019.pdf
#https://www.nature.com/articles/s41598-017-10554-0

m_e = 0.511 # Mass of electron in MeV

def gamma(ip,im): #E^2=gamma^2m^2=p^2+m^2
    return np.sqrt(1+(ip/im)**2)

def beta(ip,im): #gamma=1/sqrt(1-b^2)
    g=gamma(ip,im)
    return np.sqrt(1-1./g**2)

def betagamma(ip,im):#p=bgm
    return ip/im

def Tmax(ip,im): # Maximum energy transfer in one collision in MeV
    return 2*m_e*(ip/im)**2/(1+2*gamma(ip,im)*m_e/im+(m_e/im)**2)

def TKinheavy(ip,im): #(T+M)^2=sqrt(p)+sqrt(m)
    return np.sqrt(np.sqrt(ip)+np.sqrt(um))-im

def delta(ip,im):
    C = 4.44
    a = 0.1492
    m = 3.25
    X1 = 2.87
    X0 = 0.2014
    delta0 = 0.14
    x = np.log10(ip/im)
    #f1 = lambda x: delta0 * 10**(2*(x-X0)) # conductors pdg
    f2 = 2 * x * np.log(10) - C + (a * np.maximum(0, (X1 - x))**m) #using np.maximum to prevent warning when x > X1
    f3 = 2 * x * np.log(10) - C
    delta_full = np.where(x < X0 , 0, f2)
    delta_full = np.where(x < X1, delta_full, f3)
    return delta_full
        
def dEdxF(ip,im,iZ,zpart=1,rho=1.0,nodelta=False): #Bethe-Bloch equation
    K = 0.307075 # constant K in MeV cm mol^-1
    #rho = 2.336 # Density of material in g cm^-3 (here: silicon density)
    const   = zpart**2 * (K * rho * iZ ) / (2 * A(iZ)) * (1./beta(ip,im)**2)
    logterm = 2 * m_e * Tmax(ip,im) * ((ip/im)**2)/(I(iZ)**2) 
    dEdxV   =  const * (np.log(logterm)  - 2*(beta(ip,im))**2 - delta(ip,im))              
    if nodelta:
        print("delta:",delta(ip,im),dEdxV)
        dEdxV    =  const * (np.log(logterm) - 2*(beta(ip,im))**2)
    return dEdxV
    
mproton=938
mpion=135.4
mmuon=105.4
print("Gamma and Beta for a proton of momentum 100 MeV:",gamma(100,mproton),beta(100,mproton))
p=np.arange(15,1000000,10)
dEdxOut1p = dEdxF(p,mproton,8,1)
dEdxOut2p = dEdxF(p,mproton,18,1)
dEdxOut1pi = dEdxF(p,mpion,8,1)
dEdxOut2pi = dEdxF(p,mpion,18,1)
dEdxOut1mu = dEdxF(p,mmuon,8,1)
dEdxOut2mu = dEdxF(p,mmuon,18,1)

plt.plot(p,dEdxOut1p,label="Proton")
plt.plot(p,dEdxOut2p,label="Proton(Calcium)")

plt.plot(p,dEdxOut1pi,label="Pion")
plt.plot(p,dEdxOut2pi,label="Pion(Calcium)")

plt.plot(p,dEdxOut1mu,label="Muon")
plt.plot(p,dEdxOut2mu,label="Muon(Calcium)")

plt.xlabel('Momentum (MeV)')
plt.ylabel('dE/dx (MeV/cm) (rho=1 g/cm$^{3}$)')
plt.xscale('log')
plt.yscale('log')
plt.legend()
plt.show()


print("Beta for a proton of momentum 935 MeV (the same as its mass):",beta(935,mproton))

<h3>A Basic Simulation</h3>

We see that more energy is deposited at lower momentum, but the behavior is not monotonic. We can run a basic simulation of how the energy of a particle changes as it moves through matter by giving it some initial energy, and then stepping down the kinetic energy by subtracting the energy loss at each increment of distance. Ultimately, we want to plot both momentum and energy loss as functions of distance. We could perhaps do this analytically, but using the above formula makes it particularly nice and elegant. Let's go ahead and run our simulation for our proton energy beam!

The plot above showed energy loss as a function of momentum. However, in the following, when we say we have a 350 MeV proton, we actually mean the kinetic energy is 350 MeV. The momentum is given by:

$$
E_{\rm Total}^2 = \left (E_{\rm kin} + m \right )^2  = p^2 + m^2 \\
p^2 = (E_{kin} + m)^2 - m^2
$$

As mentioned above, the factors of $c^2$ attached to the mass have been omitted.

If the particle is nonrelativistic ($E_{\rm kin}\ll m$), this reduces to:

$$
p=\sqrt{2 E_{\rm kin} m}=mv
$$

In [ ]:
#>>>RUN: L20.4-runcell03

def dP(dE,ip,im): #solving 
    #dp = ip - np.sqrt(dE**2+ip**2-2*dE*np.sqrt(ip**2+im**2))
    #E=p^2/2m=> p=\sqrt(2mE)=>dp=sqrt(2m)/sqrt(E) dE
    #return dE*(np.sqrt(ip**2+im**2)/ip)*gamma(ip,im)
    return dE#*(ip/im)

def eToP(iE,im):
    return np.sqrt((iE+im)**2-im**2)

def sim(ie=500,im=935,idt=1e-11,iZ=8):
    xstep  = np.array([])
    estep  = np.array([])
    pstep  = np.array([])
    c=3e10
    dist=0
    e=ie
    while e > 5:
        p = eToP(e,im)
        dEdxS  = dEdxF(p,im,iZ=iZ,rho=1.06)
        #print(dEdxS)
        dx     = beta(p,im)*c*idt#speed of light
        #print(dEdxS,dP(dEdxS*dx,p,im))
        e      -= dEdxS*dx
        dist   += dx
        xstep  = np.append(xstep,dist)
        estep  = np.append(estep,dEdxS*dx)
        pstep  = np.append(pstep,e)
    return xstep,pstep,estep
    
print("350 MeV Proton Momentum:",eToP(350,mproton))
xstep150,pstep150,estep150 = sim(ie=150,im=mproton,idt=1e-11,iZ=8)
xstep200,pstep200,estep200 = sim(ie=200,im=mproton,idt=1e-11,iZ=8)
xstep250,pstep250,estep250 = sim(ie=250,im=mproton,idt=1e-11,iZ=8)
xstep300,pstep300,estep300 = sim(ie=300,im=mproton,idt=1e-11,iZ=8)

plt.plot(xstep150,pstep150,label='150 MeV')
plt.plot(xstep200,pstep200,label='200 MeV')
plt.plot(xstep250,pstep250,label='250 MeV')
plt.plot(xstep300,pstep300,label='300 MeV')
plt.xlabel('Distance(cm)')
plt.ylabel('Momentum(MeV)')
plt.legend()
plt.show()
plt.plot(xstep150,estep150,label='150 MeV')
plt.plot(xstep200,estep200,label='200 MeV')
plt.plot(xstep250,estep250,label='250 MeV')
plt.plot(xstep300,estep300,label='300 MeV')
plt.xlabel('Distance(cm)')
plt.ylabel('E-deposit(MeV/mm)')
plt.legend()
plt.show()

#sim(60,dt=1e-12)

Now, what you see is that we get a very sharp increase in the energy deposit in a relatively narrow region. As a result, we can use proton beams for cancer therapy by depositing a huge amount of energy in the body at a specific location. This peak is known as the **Bragg peak.** You can see that different beam energies can be selected in order to target different locations in the body.

<h3>More Realistic Simulations</h3>

Let's go one step further and make our simulation even more realistic. There are three ways to do this:
1. Sample the full energy profile interaction as the particle moves through the material.
2. Smear the initial beam energy by the input resolution.
3. Add other effects, in particular multiple scattering.

Let's go ahead and start with the first option, since this is often the most illustrative.

The energy loss over a particular step is not exactly the unique fixed amount given by the Bethe-Bloch formula but, instead, is actually governed by a Landau distribution, whose probability (for a mean of zero and a specific scale factor) is given by

$$
p(x|\mu,c) = \frac{1}{c\pi}\int_{0}^{\infty} e^{-t}\cos\left(t\left(\frac{x-\mu}{c}\right) + \frac{2t}{\pi}\log\left(\frac{t}{c}\right)\right) dt
$$

where $\mu$ is the most probable value (MPV) and $c$ is a width parameter. In the code below we label the mean as `landauMPV`, the width term $c$ is labeled as constant. These terms are extracted from the Physics particle data-group, and are well studied. 

To model energy loss more accurately, what we need to do is sample a Landau distribution at every step and use that sampled value in our calculation of $dE/dx$. Let's first go ahead and plot the probability density function of energy loss using the Landau distribution, and then come up with a scheme to sample it.


First lets plot the landaus.


In [ ]:
#>>>RUN: L20.4-runcell04

import pylandau
from landaupy import landau

def landauMPV(ip,im,iZ,irho=1,zpart=1):
    K = 0.307075 # constant K in MeV cm mol^-1
    const   = zpart**2 * (K * irho * iZ ) / (2 * A(iZ)) * (1./beta(ip,im)**2)
    #logterm  = 2 * m_e * Tmax(ip,im) * ((ip/im)**2)/(I(iZ)**2) 
    logterm1 = 2 * m_e *               ((ip/im)**2)/(I(iZ)) 
    logterm2 = const/I(iZ)
    dEdxV    =  const * (np.log(logterm1) + np.log(logterm2) + 0.2     - (beta(ip,im))**2 - delta(ip,im))       # 
    return dEdxV,const

def plotLandau(ip,im,idx,iZ=14,irho=1,zpart=1):
    lP=eToP(ip,im)
    lMPV,lWMPV = landauMPV(lP,im,iZ,irho,zpart)
    lMPV*=idx; lWMPV*=idx
    x=np.arange(0,15.5,0.01)
    landpy=landau.pdf(x,lMPV,lWMPV)
    return x,landpy

x,landZ150=plotLandau(150,mproton,1.0,iZ=8,irho=1.0)
x,landZ200=plotLandau(200,mproton,1.0,iZ=8,irho=1.0)
x,landZ250=plotLandau(250,mproton,1.0,iZ=8,irho=1.0)
x,landZ300=plotLandau(300,mproton,1.0,iZ=8,irho=1.0)

plt.plot(x,landZ150,label='E=150 MeV')
plt.plot(x,landZ200,label='E=200 MeV')
plt.plot(x,landZ250,label='E=250 MeV')
plt.plot(x,landZ300,label='E=300 MeV')
plt.xlabel("dE/dx (MeV/cm)/rho")
plt.ylabel("pdf")
plt.legend()
plt.show()


Now, what we can do to add some level of realism as we move along our path of the proton going through matter. Every time we step, we can sample the landau distribution pdf and step our energy loss. Given that we are sampling, it now makes sense to run this experiment a number of times. This now starts to give us a real Monte Carlo description!  

Sampling complicated distributions like this can often be slow. This code was implemented <a href="https://github.com/SengerM/landaupy/blob/main/landaupy/samplers.py#L42" target="_blank">here</a>. The strategy is to randomly sample a number from 0 to 1 to get a p-value, then you can get the x-value by inverting the CDF since we know that

$$
\rm{cdf}(x)=\int_{-\infty}^{x} p(x^{\prime}) dx^{\prime}
$$

So, we can treat the cdf as a 1-to-1 map, where the $y$ axis is probability and the $x$ axis is the pdf value integrated from $-\infty$ up to $x$. By inverting the cdf, i.e. inverting the function so that ${\rm cdf}^{-1}(y)=x$, we are effectively treating this as a lookup table.

We will stop our proton beam once its kinetic energy is less than 5 MeV. We will take a timestep of 10 picoseconds, which is roughly 1% of the total travel time. Furthemore more,  following what is shown on slide 9 in this <a href="https://indico.cern.ch/event/654712/contributions/2666034/attachments/1531773/2397743/SJ_STFCDetectors_ProCal_25-09-17.pdf" target="_blank">talk.</a> we expect a resolutoin of roughly 1.5%, which motviates the choice of the time step. 


In [ ]:
#>>>RUN: L20.4-runcell05

def simSample(ie=500,im=935,idt=1e-11,iZ=8):
    xstep  = np.array([])
    estep  = np.array([])
    pstep  = np.array([])
    c=3e10
    dist=0
    e=ie
    while e > 5:
        p = eToP(e,im)
        lMPV,lWMPV  = landauMPV(p,im,iZ=iZ,irho=1.06)
        dE     = landau.sample(lMPV, lWMPV, 1)
        dx     = beta(p,im)*c*idt#speed of light
        e      -= dE*dx
        dist   += dx
        xstep  = np.append(xstep,dist)
        estep  = np.append(estep,dE*dx)
        pstep  = np.append(pstep,e)
    return xstep,pstep,estep

xstep150,pstep150,estep150 = simSample(ie=150,im=mproton,idt=1e-11,iZ=8)
xstep200,pstep200,estep200 = simSample(ie=200,im=mproton,idt=1e-11,iZ=8)
xstep250,pstep250,estep250 = simSample(ie=250,im=mproton,idt=1e-11,iZ=8)
xstep300,pstep300,estep300 = simSample(ie=300,im=mproton,idt=1e-11,iZ=8)

plt.plot(xstep150,pstep150,label='150 MeV')
plt.plot(xstep200,pstep200,label='200 MeV')
plt.plot(xstep250,pstep250,label='250 MeV')
plt.plot(xstep300,pstep300,label='300 MeV')
plt.xlabel('Distance(cm)')
plt.ylabel('Momentum(MeV)')
plt.legend()
plt.show()
plt.plot(xstep150,estep150,label='150 MeV')
plt.plot(xstep200,estep200,label='200 MeV')
plt.plot(xstep250,estep250,label='250 MeV')
plt.plot(xstep300,estep300,label='300 MeV')
plt.xlabel('Distance(cm)')
plt.ylabel('E-deposit(MeV/mm)')
plt.legend()
plt.show()



You can observe two differences between these plots and the ones without the Landau effect. First, the curves of energy loss versus momentum look less smooth as a result of some values sampled from the Landau probability being larger or smaller than given by the exact Bethe-Bloch formula.

The much more dramatic effect is the spikes in the energy deposition as a function of distance. These occur because the Landau probability distribution has a very long tail on the high energy side. As a result, while it's still true that the energy deposition is largest at the very end, this difference is reduced somewhat.


Let's step through this more realistic version and look at the distribution of energy deposited in our system. We will run our simulation 100 times for a proton with a kinetic energy of 150 MeV and see how much the energy deposit varies, as well as how much the length of propagation varies.

For this, we will define a few observables. Let's look at following:

(1) Distance: The total distance that the particle travels\
(2) E-deposit Front: The total energy deposited in the first 3 cm of travel\
(3) E-deposit Back:  The total energy deposited in the last 3 cm of travel

In [ ]:
#>>>RUN: L20.4-runcell06

def observables(iXArr,iPArr,iEArr):
    lX=iXArr[-1]
    dEEnd=np.sum(iEArr[(iXArr > iXArr[-1]-3)])
    dEFrt=np.sum(iEArr[(iXArr < 3)])
    return lX,dEEnd,dEFrt  
    
def simNSamples(ie=150,im=mproton,iN=100,idt=1e-11,iZ=8):
    pXArr = np.array([])
    pdEBArr = np.array([])
    pdEFArr = np.array([])
    for i0 in range(iN):
        if i0 % 25 == 0:
            print(i0)
        pXstep,pPstep,pEstep = simSample(ie=ie,im=im,idt=idt,iZ=iZ)
        pX,pdEEnd,pdEFrt = observables(pXstep,pPstep,pEstep)
        pXArr   = np.append(pXArr,  pX)
        pdEBArr = np.append(pdEBArr,pdEEnd)
        pdEFArr = np.append(pdEFArr,pdEFrt)


    plt.hist(pXArr,label='Distance')
    plt.xlabel('Distance')
    plt.ylabel('N')
    plt.legend()
    plt.show()
    
    plt.hist(pdEFArr,label='dE/dx Front')
    plt.xlabel('E-Deposit Front')
    plt.ylabel('N)')
    plt.legend()
    plt.show()

    plt.hist(pdEBArr,label='dE/dx Back')
    plt.xlabel('E-Deposit Back')
    plt.ylabel('N)')
    plt.legend()
    plt.show()


#plt.plot(xstep150,estep150,label='150 MeV')
#plt.plot(xstep200,estep200,label='200 MeV')
#plt.plot(xstep250,estep250,label='250 MeV')
#plt.plot(xstep300,estep300,label='300 MeV')
#plt.xlabel('Distance(cm)')
#plt.ylabel('E-deposit(MeV/mm)')
#plt.legend()
#plt.show()

simNSamples()


As expected, there are fluctuations in the deposited energy, in particular on the higher energy side and the energy deposited at the end is a factor of 3-4 higher than at the beginning. Perhaps surprisingly, the total distance traveled is a relatively narrow peak, varying by lass than 10%.

However, running that simulation was kind of slow. Can we speed up this process?

The way we will do this is we will step through 500 simulations all at the same time in parallel. The one thing that is tricky is that sampling the Landau in parallel is not supported, so we will have to add a `for` loop to do that bit. Otherwise, we can run everything else in parallel. Let's use that extra speed to run 500 samples of 4 different values of the kinetic energy for a total of 20 times more than we did with the previous code.


In [ ]:
#>>>RUN: L20.4-runcell07

def simNParallelSample(iN, ie=500,im=935,idt=1e-10,iZ=8):
    xstep  = np.empty((0,iN))
    estep  = np.empty((0,iN))
    pstep  = np.empty((0,iN))
    c=3e10
    dist=np.zeros(iN)
    e=np.ones(iN)*ie
    print("Scanning:",ie)
    while np.any(e > 5):
        p = eToP(e,im)
        lMPV,lWMPV  = landauMPV(p,im,iZ=iZ,irho=1.06)
        dE = np.zeros(lMPV.shape)
        ##Here we have to parallelize by hand, this is not good
        for i0, (pMPV,pWMPV) in enumerate(zip(lMPV,lWMPV)):
            dE[i0]     = landau.sample(pMPV, pWMPV,1)
        dx     = beta(p,im)*c*idt#speed of light
        pdEdX  = np.minimum(dE*dx,e-0.1)
        e      -= pdEdX
        dist   += dx
        xstep  = np.vstack((xstep,dist))
        estep  = np.vstack((estep,pdEdX))
        pstep  = np.vstack((pstep,e))
    xstep = xstep.T
    estep = estep.T
    pstep = pstep.T
    return xstep,pstep,estep

def simNSamples(ie=100,im=mproton,iN=500,idt=5e-11,iZ=8):
    xstep,pstep,estep=simNParallelSample(iN,ie=ie,im=im,idt=idt,iZ=iZ)
    plt.hist(xstep[:,-1],alpha=0.5)
    plt.show()
    efront=np.zeros(xstep.shape[0])
    eback =np.zeros(xstep.shape[0])
    for i0 in range(xstep.shape[0]):
        efront[i0] = np.sum(estep[i0,xstep[i0] < 3])/3.
        eback[i0]  = np.sum(estep[i0,xstep[i0] > xstep[i0]-3])/3.
    xrange=np.arange(0,150,2.5)
    _,bins,_=plt.hist(efront,bins=xrange,alpha=0.5,label='dE/dx Front')
    plt.hist(eback, bins=bins,alpha=0.5,label='dE/dx Back')
    plt.legend()
    plt.show()
    
def sumEstep(estep,xstep):
    efront=np.zeros(xstep.shape[0])
    eback =np.zeros(xstep.shape[0])
    for i0 in range(xstep.shape[0]):
        efront[i0] = np.sum(estep[i0,xstep[i0] < 3])/3.
        #print(xstep[i0] < 3,xstep[i0] > xstep[i0,-1]-3,xstep[i0,-1]-3,xstep[i0],estep[i0])
        eback[i0]  = np.sum(estep[i0,xstep[i0] > xstep[i0,-1]-3])/3.
    return efront,eback

xstep150,pstep150,estep150=simNParallelSample(ie=150,im=mproton,iN=500,idt=1e-10,iZ=8)
xstep200,pstep200,estep200=simNParallelSample(ie=200,im=mproton,iN=500,idt=1e-10,iZ=8)
xstep250,pstep250,estep250=simNParallelSample(ie=250,im=mproton,iN=500,idt=1e-10,iZ=8)
xstep300,pstep300,estep300=simNParallelSample(ie=300,im=mproton,iN=500,idt=1e-10,iZ=8)

plt.hist(xstep150[:,-1],alpha=0.5,label='E=150 MeV')
plt.hist(xstep200[:,-1],alpha=0.5,label='E=200 MeV')
plt.hist(xstep250[:,-1],alpha=0.5,label='E=250 MeV')
plt.hist(xstep300[:,-1],alpha=0.5,label='E=300 MeV')
plt.xlabel('Distance')
plt.ylabel('N')
plt.legend()
plt.show()

ef150,eb150=sumEstep(estep150,xstep150)
ef200,eb200=sumEstep(estep200,xstep200)
ef250,eb250=sumEstep(estep250,xstep250)
ef300,eb300=sumEstep(estep300,xstep300)

xrange=np.arange(0,10,0.25)
plt.hist(ef150,bins=xrange,alpha=0.5,label='E=150 MeV (dE/dx Front)')
plt.hist(ef200,bins=xrange,alpha=0.5,label='E=200 MeV (dE/dx Front)')
plt.hist(ef250,bins=xrange,alpha=0.5,label='E=250 MeV (dE/dx Front)')
plt.hist(ef300,bins=xrange,alpha=0.5,label='E=300 MeV (dE/dx Front)')
plt.xlabel('E-Deposit Front')
plt.ylabel('N)')
plt.legend()
plt.show()

xrange=np.arange(0,60,2)
plt.hist(eb150,bins=xrange,alpha=0.5,label='E=150 MeV (dE/dx Back)')
plt.hist(eb200,bins=xrange,alpha=0.5,label='E=200 MeV (dE/dx Back)')
plt.hist(eb250,bins=xrange,alpha=0.5,label='E=250 MeV (dE/dx Back)')
plt.hist(eb300,bins=xrange,alpha=0.5,label='E=300 MeV (dE/dx Back)')
plt.xlabel('E-Deposit Back')
plt.ylabel('N)')
plt.legend()
plt.show()


From these distributions, we can start to make interesting conclusions. As expected, the higher energy beams tend to go farther. They also deposit slightly less energy in the front, but deposit about the same as the lower energies in the back. Notice also the 150 MeV feature where a non-trivial fraction of the protons deposit a lot more than the average in the first 3 cm.

This Monte Carlo we have developed is pretty sophisticated, but we can do even better if we take into account that our beam is a full 3D distribution. We will consider this next!


<a name='exercises_21_4'></a>     

| [Top](#section_21_0) | [Restart Section](#section_21_4) | [Next Section](#section_21_5) |


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Ex-20.4.1</span>

In the preceding section, we used `sumEstep` to determine the total amount of energy deposited in the first and last 3 cm of a particle's path, and created a histogram. Now, write a function `sumEstepInWindow` that performs a similar action within a window of width `w`, centered at a position `d`.

What is the total energy deposited in a window of width 3 cm, centered at a distance of 5 cm, for an initial energy of 250 MeV? Report your answer as a number in MeV with precision 0.1 MeV.

<br>

In [ ]:
#>>>EXERCISE: L20.4.1

def sumEstepInWindow(estep, xstep, d, w):
    ewindow = np.zeros(xstep.shape[0])
    #YOUR CODE HERE
    return ewindow

# Define the window width w and center distance d
w = 3  # cm, change this value as needed
d = 5  # cm, change this value as needed

# Calculate total energy deposited in the window of width w cm centered at distance d cm for each initial energy
total_deposited_150 = np.round(np.mean(sumEstepInWindow(estep150, xstep150, d, w)),1)
total_deposited_200 = np.round(np.mean(sumEstepInWindow(estep200, xstep200, d, w)),1)
total_deposited_250 = np.round(np.mean(sumEstepInWindow(estep250, xstep250, d, w)),1)
total_deposited_300 = np.round(np.mean(sumEstepInWindow(estep300, xstep300, d, w)),1)

print(f'Total energy deposited in a {w} cm window centered at {d} cm for 150 MeV: {total_deposited_150} MeV')
print(f'Total energy deposited in a {w} cm window centered at {d} cm for 200 MeV: {total_deposited_200} MeV')
print(f'Total energy deposited in a {w} cm window centered at {d} cm for 250 MeV: {total_deposited_250} MeV')
print(f'Total energy deposited in a {w} cm window centered at {d} cm for 300 MeV: {total_deposited_300} MeV')

### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Ex-20.4.2</span>

Now, we will use the code from the preceding exercise to obtain the total energy deposited within a 3 cm window as a function of distance. A plot of energy deposited versus distance will yield the Bragg peak for a given initial energy! Create this plot for all 4 initial energies that we have been considering. For your answer to this exercise, find the location of the Bragg peak for the 250 MeV protons. Enter your answer as a number in centimeters with precision $\pm 0.5$.

<br>

In [ ]:
#>>>EXERCISE: L20.4.2

# Define the window width w
w = 3  # cm, change this value as needed

# Define the range of d values
d_values = np.arange(0, 60, 0.5)  # cm, change this range as needed

# Initialize arrays to store total energy deposited for each initial energy and each d value
total_deposited_150 = []
total_deposited_200 = []
total_deposited_250 = []
total_deposited_300 = []

# Calculate total energy deposited in the window for each d value
#YOUR CODE HERE

# Plot the results
#YOUR CODE HERE

# Find the highest peak for each energy
#YOUR CODE HERE
    
# Print peak information
#YOUR CODE HERE

### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Ex-20.4.3</span>

Now, create a plot of beam energy as a function of depth of maximum energy deposition. In other words, determine the Bragg peak for a given beam energy using the tools above, and then create a plot of beam energy vs. Bragg peak location. You can use the same 4 starting kinetic energies considered previously. You may find it helpful to use the `interp1d` function in the `scipy.interpolate` library.

Approximately what beam energy is required to make its maximum energy deposit at a depth of 25 cm? Enter your answer as a number in MeV with precision $\pm 10$.

<br>

In [ ]:
#>>>EXERCISE: L20.4.3

# Define the range of initial energies
initial_energies = [150, 200, 250, 300]
peak_distances = []#YOUR CODE HERE


# Plot the distance of the peak vs initial energy
plt.plot(peak_distances, initial_energies, marker='o')
plt.ylabel('Initial Energy (MeV)')
plt.xlabel('Depth of Max Energy Deposit (cm)')
plt.title('Initial Energy vs Depth of Max Energy Deposit')
plt.grid(True)
plt.show()


# Interpolate the data to find the initial energy required for a desired penetration depth
from scipy.interpolate import interp1d

desired_depth = 25  # cm, change this value as needed

#YOUR CODE HERE

<a name='section_21_5'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L20.5 Bragg Scattering for Proton Therapy in Multiple Dimensions </h2>  

| [Top](#section_21_0) | [Previous Section](#section_21_4) | [Exercises](#exercises_21_5) |

*The material in this section is discussed in the video **<a href="https://courses.mitxonline.mit.edu/learn/course/course-v1:MITxT+8.S50.3x+3T2023/block-v1:MITxT+8.S50.3x+3T2023+type@sequential+block@seq_LS20/block-v1:MITxT+8.S50.3x+3T2023+type@vertical+block@vert_LS20_vid5" target="_blank">HERE</a>.** You are encouraged to watch that video and use this notebook concurrently.*

<h3>Overview</h3>

In addition to losing energy, the proton beam will also scatter in different directions as it moves through the body. The scattering occurs from the fact that the proton is charged, and the nuclei of the atoms in the material are charged, and so we have Coulomb scattering (aka Rutherford scattering)of the proton from the nuclei. This is an elastic process that will inevitably spread the beam out in the transverse direction. Because of the very large mass difference, the collisions of protons with electrons (which is responsible for the energy loss) does not cause appreciable scattering. In contrast, the nuclei are much heavier than the proton so those collisions cause scattering but not much energy loss. The scattering angle is often written as:

$$
\sigma_{\theta} = \frac{13.6 \rm MeV}{\beta cp} z \sqrt{\frac{x}{X_{0}}}\left(1+0.038\log\left(\frac{xz^{2}}{X_{0}\beta^{2}}\right)\right)
$$

where $z$ is the particle charge $X_{0}(z)$ is the electron density of the material, known as the radaition length, $\beta$ is the velocity particles divided by the speed of light, $c$, $p$ is the momentum. Finally $x$ is the distance traversed. The more distance, the larger the scatter. Lastly, to get $X_{0}$ we will just use a formula. Suffice it to say its a function of the nuclear charge $Z$, I won't go into the details but you can read more about $X_{0}$ and the formula above <a href="https://pdg.lbl.gov/2022/reviews/rpp2022-rev-passage-particles-matter.pdf" target="_blank"> here</a>. 

We can simplify things since the actual properties of this scattering have been studied experimentally and the results can be fit with a parameterization containing the following two terms:

$$
\Delta y = z_{1} \Delta x \frac{\sigma_{\theta}}{\sqrt{12}} + \frac{z_{2} \Delta x\sigma_{\theta}}{2}\\
\Delta \theta     = z_{2} \sigma_{\theta}
$$

where $\Delta x$ is the distance traveled through the material, while $\Delta y$ and $\Delta \theta$ are the y-scatter  of the beam in the transverse direction to its motion and the scatter of the angular deflection of the beam, respectively, The quantities $z_{1}$ and $z_{2}$ are two normal randomly sampled distributions with $\sigma=1$ that are independent of one another.

Lets go ahead and plot these. Note that the first block below is just the code from above, so that we can quickly load it here. 

In [ ]:
#>>>RUN: L20.5-runcell00

#redefining relevant data and functions from above

def A(iZ,iPlot=False):
    #https://github.com/nrc-cnrc/EGSnrc/blob/master/HEN_HOUSE/pegs4/pegs4.mortran#L1354-L1391
    lA=[1.00797,4.0026,6.939,9.0122,10.811,12.01115,14.0067,
     15.9994,18.9984,20.183,22.9898,24.312,26.9815,28.088,30.9738,
     32.064,35.453,39.948,39.102,40.08,44.956,47.90,50.942,51.998,
     54.9380,55.847,58.9332,58.71,63.54,65.37,69.72,72.59,74.9216,
     78.96,79.808,83.80,85.47,87.62,88.905,91.22,92.906,95.94,99.0,
     101.07,102.905,106.4,107.87,112.4,114.82,118.69,121.75,127.60,
     126.9044,131.30,132.905,137.34,138.91,
     140.12,140.907,144.24,147.,150.35,151.98,157.25,158.924,162.50,
     164.930,167.26,168.934,173.04,174.97,178.49,180.948,183.85,
     186.2,190.2,192.2,195.08,196.987,200.59,204.37,207.19,208.980,
     210.,210.,222.,223.,226.,227.,232.036,231.,238.03,237.,242.,
     243.,247.,247.,248.,254.,253.
    ]
    lZ=np.arange(1,len(lA)+1)
    if iPlot:
        plt.plot(lZ,lA/lZ)
        plt.xlabel('Z')
        plt.ylabel('A/Z (Atomic mass/Z)')
        plt.show()
    return lA[iZ-1]

def beta(ip,im): #gamma=1/sqrt(1-b^2)
    g=gamma(ip,im)
    return np.sqrt(1-1./g**2)

def gamma(ip,im): #E^2=gamma^2m^2=p^2+m^2
    return np.sqrt(1+(ip/im)**2)

def I(iZ,iPlot=False):
    #https://github.com/nrc-cnrc/EGSnrc/blob/master/HEN_HOUSE/pegs4/pegs4.mortran#L1354-L1391
    lI=[19.2,41.8,40.,63.7,76.0,78.0,82.0,95.0,115.,137.,
     149.,156.,166.,173.,173.,180.,174.,188.,190.,191.,216.,233.,245.,
     257.,272.,286.,297.,311.,322.,330.,334.,350.,347.,348.,357.,352.,
     363.,366.,379.,393.,417.,424.,428.,441.,449.,470.,470.,469.,488.,
     488.,487.,485.,491.,482.,488.,491.,501.,523.,535.,546.,560.,574.,
     580.,591.,614.,628.,650.,658.,674.,684.,694.,705.,718.,727.,736.,
     746.,757.,790.,790.,800.,810.,823.,823.,830.,825.,794.,827.,826.,
     841.,847.,878.,890.,902.,921.,934.,939.,952.,966.,980.,994.]
    lZ=np.arange(1,len(lI)+1)
    if iPlot:
        plt.plot(lZ,lI/lZ)
        plt.xlabel('Z')
        plt.ylabel('I$_{adj}$/Z (eV/Z)')
        plt.show()
    return lI[iZ]*1e-6 #MeV not eV

def eToP(iE,im):
    return np.sqrt((iE+im)**2-im**2)

def delta(ip,im):
    C = 4.44
    a = 0.1492
    m = 3.25
    X1 = 2.87
    X0 = 0.2014
    delta0 = 0.14
    x = np.log10(ip/im)
    #f1 = lambda x: delta0 * 10**(2*(x-X0)) # conductors pdg
    f2 = 2 * x * np.log(10) - C + (a * np.maximum(0, (X1 - x))**m) #using np.maximum to prevent warning when x > X1
    f3 = 2 * x * np.log(10) - C
    delta_full = np.where(x < X0 , 0, f2)
    delta_full = np.where(x < X1, delta_full, f3)
    return delta_full

m_e = 0.511 # Mass of electron in MeV

def landauMPV(ip,im,iZ,irho=1,zpart=1):
    K = 0.307075 # constant K in MeV cm mol^-1
    const   = zpart**2 * (K * irho * iZ ) / (2 * A(iZ)) * (1./beta(ip,im)**2)
    #logterm  = 2 * m_e * Tmax(ip,im) * ((ip/im)**2)/(I(iZ)**2)
    logterm1 = 2 * m_e *               ((ip/im)**2)/(I(iZ))
    logterm2 = const/I(iZ)
    dEdxV    =  const * (np.log(logterm1) + np.log(logterm2) + 0.2     - (beta(ip,im))**2 - delta(ip,im))       #
    return dEdxV,const

def sumEstep(estep,xstep):
    efront=np.zeros(xstep.shape[0])
    eback =np.zeros(xstep.shape[0])
    for i0 in range(xstep.shape[0]):
        efront[i0] = np.sum(estep[i0,xstep[i0] < 3])/3.
        #print(xstep[i0] < 3,xstep[i0] > xstep[i0,-1]-3,xstep[i0,-1]-3,xstep[i0],estep[i0])
        eback[i0]  = np.sum(estep[i0,xstep[i0] > xstep[i0,-1]-3])/3.
    return efront,eback

Now, we compute $X_{0}$ and $\sigma_\theta$ and finally the scatter. Lastly, we will plot some of these formula, to just see how they look. 

In [ ]:
#>>>RUN: L20.5-runcell01

def X0(iZ):
    const=(716.408**-1)/A(iZ)
    a = iZ/137.
    Lrad =np.log(184.15*iZ**(-1./3.))
    Lradp=np.log(1194*iZ**(-2./3.))
    fZ = a**2*((1+a**2)**(-1)+0.20206-0.0369*a**2+0.0083*a**4-0.002*a**6)
    val=const*(iZ**2*(Lrad-fZ)+iZ*Lradp)
    return 1./val
    
def sigmaTheta(ip,im,iX0,idx=1.0,zpart=1):
    C=13.6
    X0=iX0
    dx=idx/iX0
    const=C/(beta(ip,im)*ip)*zpart*np.sqrt(dx)
    logterm=1+0.038*np.log(dx*zpart**2/beta(ip,im)**2)
    return const*logterm

def thetaScatter(ip,im,iX0,idx,zpart=1):
    z1=np.random.normal(0,1,ip.shape[0])
    z2=np.random.normal(0,1,ip.shape[0])
    stheta=sigmaTheta(ip,im,iX0,zpart)
    dy    =z1*idx*stheta/np.sqrt(12.) + z2*idx*stheta/2 
    dtheta=z2*stheta
    return dtheta,dy

lZ=np.arange(1,100,1)
lX0=np.zeros(len(lZ))
for pZ in lZ:
    lX0[pZ-1] = X0(pZ)
print(X0(82))
plt.plot(lZ,lX0)
plt.xlabel("Z")
plt.ylabel("$X_{0}$")
plt.ylim(0,100)
plt.show()

lP=np.arange(10,500,1)
mproton=938  # Mass of proton in MeV
lST = sigmaTheta(lP,mproton,X0(8))
plt.plot(lP,lST)
plt.yscale('log')
plt.xlabel('P (MeV)')
plt.ylabel('$\sigma_{\Theta}$')
plt.show()

What we see above is that we have very large scattering angles $\sigma_{\theta}$ when we get to low momenta. Basically, the proton can go anywhere and this can spread out our beam. If we are using this for cancer therapy, it can cause more extensive damage than we intend.

One important thing to consider is that we need to make sure our angular scatter is physical, that means we should make sure that our angles are OK. Note that the following code takes quite a while to run.


In [ ]:
#>>>RUN: L20.5-runcell02

def simNYParallelSample(iN, ie=500,im=935,idt=1e-10,iZ=8):
    xstep  = np.empty((0,iN))
    ystep  = np.empty((0,iN))
    estep  = np.empty((0,iN))
    pstep  = np.empty((0,iN))
    theta=0
    y=0
    c=3e10
    dist=np.zeros(iN)
    e=np.ones(iN)*ie
    lX0 = X0(iZ)
    print("Scanning:",ie)
    while np.any(e > 5):
        p = eToP(e,im)
        lMPV,lWMPV  = landauMPV(p,im,iZ=iZ,irho=1.06)
        dE = np.zeros(lMPV.shape)
        ##Here we have to parallelize by hand, this is not good
        for i0, (pMPV,pWMPV) in enumerate(zip(lMPV,lWMPV)):
            dE[i0]     = landau.sample(pMPV, pWMPV,1)
        dx     = beta(p,im)*c*idt#speed of light
        dTheta,dy = thetaScatter(p,im,lX0,idx=dx,zpart=1)
        pdEdX  = np.minimum(dE*dx,e-0.1)
        e      -= pdEdX
        dist   += dx*np.cos(theta)
        y      += dy + np.sin(theta)*dx
        theta  += dTheta
        xstep  = np.vstack((xstep,dist))
        ystep  = np.vstack((ystep,y))
        estep  = np.vstack((estep,pdEdX))
        pstep  = np.vstack((pstep,e))        
    xstep = xstep.T
    estep = estep.T
    pstep = pstep.T
    ystep = ystep.T
    return xstep,pstep,estep,ystep

xstep150,pstep150,estep150,ystep150=simNYParallelSample(ie=150,im=mproton,iN=1000,idt=1e-10,iZ=8)
xstep200,pstep200,estep200,ystep200=simNYParallelSample(ie=200,im=mproton,iN=1000,idt=1e-10,iZ=8)
xstep250,pstep250,estep250,ystep250=simNYParallelSample(ie=250,im=mproton,iN=1000,idt=1e-10,iZ=8)
xstep300,pstep300,estep300,ystep300=simNYParallelSample(ie=300,im=mproton,iN=1000,idt=1e-10,iZ=8)

xrange=np.arange(0,60,2)
plt.hist(xstep150[:,-1],bins=xrange,alpha=0.5,label='E=150 MeV')
plt.hist(xstep200[:,-1],bins=xrange,alpha=0.5,label='E=200 MeV')
plt.hist(xstep250[:,-1],bins=xrange,alpha=0.5,label='E=250 MeV')
plt.hist(xstep300[:,-1],bins=xrange,alpha=0.5,label='E=300 MeV')
plt.xlabel('x-Distance')
plt.ylabel('N')
plt.legend()
plt.show()

xrange=np.arange(-10,10,0.25)
plt.hist(ystep150[:,-1],bins=xrange,alpha=0.5,label='E=150 MeV')
plt.hist(ystep200[:,-1],bins=xrange,alpha=0.5,label='E=200 MeV')
plt.hist(ystep250[:,-1],bins=xrange,alpha=0.5,label='E=250 MeV')
plt.hist(ystep300[:,-1],bins=xrange,alpha=0.5,label='E=300 MeV')
plt.xlabel('y-Distance')
plt.ylabel('N')
plt.legend()
plt.show()

ef150,eb150=sumEstep(estep150,xstep150)
ef200,eb200=sumEstep(estep200,xstep200)
ef250,eb250=sumEstep(estep250,xstep250)
ef300,eb300=sumEstep(estep300,xstep300)

xrange=np.arange(0,15,0.5)
plt.hist(ef150,bins=xrange,alpha=0.5,label='E=150 MeV (dE/dx Front)')
plt.hist(ef200,bins=xrange,alpha=0.5,label='E=200 MeV (dE/dx Front)')
plt.hist(ef250,bins=xrange,alpha=0.5,label='E=250 MeV (dE/dx Front)')
plt.hist(ef300,bins=xrange,alpha=0.5,label='E=300 MeV (dE/dx Front)')
plt.xlabel('E-Deposit Front')
plt.ylabel('N)')
plt.legend()
plt.show()

xrange=np.arange(0,60,2)
plt.hist(eb150,bins=xrange,alpha=0.5,label='E=150 MeV (dE/dx Back)')
plt.hist(eb200,bins=xrange,alpha=0.5,label='E=200 MeV (dE/dx Back)')
plt.hist(eb250,bins=xrange,alpha=0.5,label='E=250 MeV (dE/dx Back)')
plt.hist(eb300,bins=xrange,alpha=0.5,label='E=300 MeV (dE/dx Back)')
plt.xlabel('E-Deposit Back')
plt.ylabel('N)')
plt.legend()
plt.show()


<h3>Making a Spatial Map</h3>

Now, this is where things become really interesting. The simulation we made above models a very specific process, to which we have added more and more elements of realism. However, these additions make the code more complicated and slower. That being said, we see in the end that the distributions we model are surprisingly smooth given that we have only been running hte code for a few minutes. Sometimes, it takes hours or days to generate events to make nice smooth interpretable distributions. 

However, in the interest of being fast, what we can do to speed things up is to build a parametric model for the behavior of what is going. This is often how we try to characterize simulations. In the next lecture, we will do this by using deep learning to auto generate these profiles. What we will is something that is reasonably accurate, but almost 1000 times faster. 

In [ ]:
#>>>RUN: L20.5-runcell03

def plotImage(iId,ixstep,iestep,iystep):
    #plt.plot(ixstep[iId],iystep[iId])#,iestep[iId])
    #plt.show()
    #Now let's make a regular image 
    xbin = np.arange(-1,55, 2)
    ybin = np.arange(-3.75, 3.75, 0.25)
    #xbin = np.arange(-0.5,60.5, 1)
    #ybin = np.arange(-5.125, 5.125, 0.25)
    print("A:",len(ixstep.flatten()),len(iystep.flatten()),len(iestep.flatten()))
    H, xedges, yedges = np.histogram2d(ixstep.flatten(), iystep.flatten(), bins=(xbin, ybin),weights=iestep.flatten())  
    plt.imshow(H.T,extent=[xedges[0], xedges[-1], yedges[0], yedges[-1]])  
    plt.show()
    #X, Y = np.meshgrid(xedges, yedges)
    #plt.pcolormesh(X,Y,H.T)  
    #plt.show()

plotImage(-1,xstep150,estep150,ystep150)
plotImage(-1,xstep200,estep200,ystep200)
plotImage(-1,xstep250,estep250,ystep250)
plotImage(-1,xstep300,estep300,ystep300)

<a name='exercises_21_5'></a>     

| [Top](#section_21_0) | [Restart Section](#section_21_5) | [Next Section](#section_21_6) |


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Ex-20.5.1</span>

Construct the same plot as in the exercise `Ex-20.4.3`, but now using this more realistic scattering model. Specifically, create a plot of beam energy vs Bragg peak location. Again, determine the approximate beam energy that is required to make a maximum energy deposit at a depth of 25 cm.

How are the results of the more realistic model presented in this section different from those of the simplified model? Select ALL that apply:

A) The Bragg peaks are shifted farther out in distance.\
B) The Bragg peaks are shifted closer in distance.\
C) The Bragg peak positions are approximately unchanged.\
D) The Bragg peaks are wider.\
E) The Bragg peaks are narrower.\
F) The width of the Bragg peaks are approximately unchanged.


<br>